# Colab

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip install -U accelerate
!pip install -q transformers datasets evaluate
%cd /content/drive/MyDrive/Code/Maitrise/sappelli_email_classification

/content/drive/MyDrive/Code/Maitrise/sappelli_email_classification


# Preprocessing

In [1]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="./data/act1_encoded.csv")
dataset['train'][0]

c:\Users\alexi\miniconda3\envs\sapelli\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 0 examples [00:00, ? examples/s]c:\Users\alexi\miniconda3\envs\sapelli\Lib\site-packages\datasets\download\streaming_download_manager.py:784: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)
Generating train split: 570 examples [00:00, 9155.90 examples/s]


{'label': 3,
 'text': 'It was cute!\tGenia FitzGerald\t10/10/2000 03:23 PM\t\t \t\t To: Marie Heard/Enron Communications@Enron Communications\t\t cc: Tana Jones/HOU/ECT@ECT\t\t Subject: Re: HOTDOG !This is sooo adorable!Thanks for inviting me to share Mexican food with you.  Tana said it was YOUR fault I wasn t invited!!!!!Genia----- Forwarded by Genia FitzGerald/HOU/ECT on 10/10/2000 03:21 PM -----\t\t----- Forwarded by Genia FitzGerald/HOU/ECT on 10/10/2000 02:30 PM ----- - Hotdog PHOTO2.jpg'}

In [2]:
# prompt: Split the dataset, but keep the HuggingFace format

train_test_dataset = dataset["train"].train_test_split(test_size=0.2, shuffle=False, seed=42)

# prompt: lenght of train and test

train_length = len(train_test_dataset['train'])
test_length = len(train_test_dataset['test'])

print(f"Train length: {train_length}")
print(f"Test length: {test_length}")

Train length: 456
Test length: 114


In [3]:
train_test_dataset['test'][0]

{'label': 7,
 'text': 'I will be out tomorrow and I may leave early this afternoon.  Could you email me the deal numbers for the 3 purchases and 12 sales for the month of August?'}

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [5]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [6]:
tokenized_dataset = train_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/456 [00:00<?, ? examples/s]

Map: 100%|██████████| 114/114 [00:00<00:00, 1023.36 examples/s]


In [7]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Evaluation metrics

In [8]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Training

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=8)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results/act1",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


TrainOutput(global_step=145, training_loss=1.2340486723801185, metrics={'train_runtime': 206.4998, 'train_samples_per_second': 11.041, 'train_steps_per_second': 0.702, 'total_flos': 599103257072640.0, 'train_loss': 1.2340486723801185, 'epoch': 5.0})

In [ ]:
trainer.train()

In [ ]:
# prompt: save model with name act1

model.save_pretrained("./models/act1")


# Loading

In [9]:
# prompt: load local model in /content/drive/MyDrive/Code/Maitrise/sappelli/models/act1/model.safetensors
from transformers import AutoModelForSequenceClassification

# Download model and configuration from huggingface.co and cache.
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=8)
# Model was saved using *save_pretrained('./test/saved_model/')* (for example purposes, not runnable).
model = AutoModelForSequenceClassification.from_pretrained("./models/act1/")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Evaluation

In [15]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [16]:
from torch.utils.data import DataLoader

eval_dataloader = DataLoader(train_test_dataset["test"], batch_size=8)

In [19]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()


AttributeError: 'list' object has no attribute 'to'

In [ ]:
# prompt: evaluate the results on the test dataset

evaluation = trainer.evaluate(tokenized_dataset["test"])
print(evaluation)


NameError: name 'trainer' is not defined

In [ ]:
# prompt: evaluate using f1 score

from sklearn.metrics import f1_score

# Compute and print the F1 score
f1 = f1_score(tokenized_dataset["test"]["label"], trainer.predict(tokenized_dataset["test"]).predictions.argmax(axis=-1), average="micro")
print(f"F1 score: {f1}")


KeyError: "Column labels not in the dataset. Current columns in the dataset: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask']"